<a href="https://colab.research.google.com/github/raheelam98/langgraph_guru/blob/main/agent_basic_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
%%capture --no-stderr
%pip install -q -U google-generativeai

In [37]:
# packages:
%%capture --no-stderr
%pip install -U langgraph langsmith langchain_google_genai langchain   python-dotenv

In [45]:
# API keys set up
# Configure environment to connect to LangSmith.
import os
from google.colab import userdata

os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "agent-basic-tool"

# LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"


In [39]:
import os
from google.colab import userdata
gemini_api_key = userdata.get('GEMINI_API_KEY')

In [40]:
# llm
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    max_retries=2,
    api_key=gemini_api_key
)

In [41]:
llm.invoke("hi")

AIMessage(content='Hi there! How can I help you today?\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-087311a4-9031-4c85-ae44-0685f14b683a-0', usage_metadata={'input_tokens': 2, 'output_tokens': 11, 'total_tokens': 13, 'input_token_details': {'cache_read': 0}})

In [42]:
response = llm.invoke("hi")
print(response)

content='Hi there! How can I help you today?\n' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-4a15225d-b000-4cd8-a44c-b5610549209d-0' usage_metadata={'input_tokens': 2, 'output_tokens': 11, 'total_tokens': 13, 'input_token_details': {'cache_read': 0}}


In [18]:
# packages
%%capture --no-stderr
%pip install -U tavily-python langchain_community

In [19]:
# API keys set up
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

**Lang Chain and Lang Graph Documents**

**Agent Basic**

* ToolNode
* ToolMessage



[JsonOutputParser](https://python.langchain.com/api_reference/core/output_parsers/langchain_core.output_parsers.json.JsonOutputParser.html)

**JsonOutputParser :** Parse the output of an LLM call to a JSON object.

When used in streaming mode, it will yield partial JSON objects containing all the keys that have been returned so far.

In streaming, if diff is set to True, yields JSONPatch operations describing the difference between the previous and the current object.

In [20]:
# ToolExecutorAgent (Session 3 - Lang Graph)

from dotenv import load_dotenv

load_dotenv()

from typing import List

from pydantic import BaseModel, Field


class Reflection(BaseModel):
    missing: str = Field(description="Critique of what is missing.")
    superfluous: str = Field(description="Critique of what is superfluous")


class AnswerQuestion(BaseModel):
    """Answer the question."""

    answer: str = Field(description="~250 word detailed answer to the question.")
    reflection: Reflection = Field(description="Your reflection on the initial answer.")
    search_queries: List[str] = Field(
        description="1-3 search queries for researching improvements to address the critique of your current answer."
    )

# ToolMessage
from ast import parse
from langchain_core.messages import HumanMessage, ToolMessage, SystemMessage, BaseMessage, AIMessage
from typing import List
from langgraph.prebuilt import ToolInvocation

from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser(return_id=True)

def exeute_tools(state: List[BaseMessage])-> List[ToolMessage]:
  tool_invocation : AIMessage = state[-1]
  parsed_tool_calls = parser.invoke(tool_invocation)

  ids = []
  tool_invocation = []

  for parsed_call in parsed_tool_calls:
    for query in parsed_call["args"]["search_queries"]:
      print(query)
      tool_invocation.append(ToolInvocation(
        tool="tavily_search_results_json",
        tool_input=query
      ))
      ids.append(parsed_call["id"])
  pass

